In [ ]:
"""
Hyperparameter optimization utilizing Optuna
Code based on: https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py
"""

import sys
import pathlib
import numpy as np
import pandas as pd
import optuna
import torch
from torch.utils.data import DataLoader, TensorDataset

from betavae import BetaVAE, train_vae, evaluate_vae
from optimize_utils import get_optimize_args

In [ ]:
# Load command line arguments
args = get_optimize_args()

# Load data

output_dir = pathlib.Path("data")

sys.path.insert(0, "../0.data-download/scripts/")
from data_loader import load_train_test_data

data_directory = pathlib.Path("../0.data-download/data")
dfs = load_train_test_data(data_directory, train_or_test="all", load_gene_stats=True)

train_feat = dfs[0]
test_feat = dfs[1]
load_gene_stats = dfs[2]

# Prepare data for training
train_features_df = train_feat.drop(columns=["ModelID", "age_and_sex"])
test_features_df = test_feat.drop(columns=["ModelID", "age_and_sex"])

# create dataframe containing the genes that passed an initial QC (see Pan et al. 2022) and their corresponding gene label and extract the gene labels
gene_dict_df = pd.read_csv(
    "../0.data-download/data/CRISPR_gene_dictionary.tsv", delimiter="\t"
)
gene_list_passed_qc = gene_dict_df.query("qc_pass").dependency_column.tolist()

# create new training and testing dataframes that contain only the corresponding genes
train_df = train_feat.filter(gene_list_passed_qc, axis=1)
test_df = test_feat.filter(gene_list_passed_qc, axis=1)

In [ ]:
# Normalize data
train_data = train_df.values.astype(np.float32)
test_data = test_df.values.astype(np.float32)

# Normalize based on data distribution
train_data = (train_data - np.min(train_data, axis=0)) / (
    np.max(train_data, axis=0) - np.min(train_data, axis=0)
)
test_data = (test_data - np.min(test_data, axis=0)) / (
    np.max(test_data, axis=0) - np.min(test_data, axis=0)
)

# Convert dataframes to tensors
train_tensor = torch.tensor(train_data, dtype=torch.float32)
test_tensor = torch.tensor(test_data, dtype=torch.float32)

In [4]:
# Objective function for Optuna
def objective(trial):
    # Define hyperparameters
    latent_dim = trial.suggest_int(
        "latent_dim", args.min_latent_dim, args.max_latent_dim
    )
    beta = trial.suggest_float("beta", args.min_beta, args.max_beta)
    learning_rate = trial.suggest_categorical(
        "learning_rate", [5e-3, 1e-3, 1e-4, 1e-5, 1e-6]
    )
    batch_size = trial.suggest_categorical("batch_size", [16, 48, 80, 112])
    epoch = trial.suggest_int("epochs", args.min_epochs, args.max_epochs, step=args.epoch_step)

    # Create DataLoader
    train_loader = DataLoader(
        TensorDataset(train_tensor), batch_size=batch_size, shuffle=True
    )
    test_loader = DataLoader(
        TensorDataset(test_tensor), batch_size=batch_size, shuffle=False
    )

    model = BetaVAE(input_dim=train_df.shape[1], latent_dim=latent_dim, beta=beta)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    loss = train_vae(model, train_loader, optimizer, epochs=epoch)

    # Evaluate VAE
    val_loss = evaluate_vae(model, test_loader)

    return val_loss


# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)

Epoch 491, Loss: 55.878919795866935
Epoch 492, Loss: 55.5366160140499
Epoch 493, Loss: 55.85907506142893
Epoch 494, Loss: 55.71633635490171
Epoch 495, Loss: 55.49738104051159
Epoch 496, Loss: 55.304024540070564
Epoch 497, Loss: 55.07171115013861
Epoch 498, Loss: 55.21754626858619
Epoch 499, Loss: 55.61180983020413
Epoch 500, Loss: 55.368352798954135
Epoch 501, Loss: 55.132697241998486
Epoch 502, Loss: 55.145894578503025
Epoch 503, Loss: 54.94942823840726
Epoch 504, Loss: 54.69184538810484
Epoch 505, Loss: 54.788622338079634
Epoch 506, Loss: 54.37647189232611
Epoch 507, Loss: 54.38663830172631
Epoch 508, Loss: 54.23632229712702
Epoch 509, Loss: 54.684276044291835
Epoch 510, Loss: 54.24458401587702
Epoch 511, Loss: 54.384705101751514
Epoch 512, Loss: 54.47348620999244
Epoch 513, Loss: 54.614903840095764
Epoch 514, Loss: 54.377781943044354
Epoch 515, Loss: 53.89189252299647
Epoch 516, Loss: 54.108918260143646
Epoch 517, Loss: 53.824832409274194
Epoch 518, Loss: 54.11534404139365
Epoch 519

[I 2024-07-18 11:30:25,848] Trial 476 finished with value: 103.2959661553376 and parameters: {'latent_dim': 55, 'beta': 1.7479475503465092, 'learning_rate': 0.0001, 'batch_size': 48, 'epochs': 905}. Best is trial 221 with value: 99.45155679744525.


Epoch 903, Loss: 47.70845939390121
Epoch 904, Loss: 48.034304750504035
Epoch 0, Loss: 220.7206593371976
Epoch 1, Loss: 174.69905934979838
Epoch 2, Loss: 144.75451116746473
Epoch 3, Loss: 122.7037365328881
Epoch 4, Loss: 106.43429010206653
Epoch 5, Loss: 92.0855619172127
Epoch 6, Loss: 80.89436747889366
Epoch 7, Loss: 71.84200908045615
Epoch 8, Loss: 64.98195521200857
Epoch 9, Loss: 60.27624291204637
Epoch 10, Loss: 56.974199770035284
Epoch 11, Loss: 53.98007351783014
Epoch 12, Loss: 52.43547221522177
Epoch 13, Loss: 50.642262356665825
Epoch 14, Loss: 49.91559058404738
Epoch 15, Loss: 49.45923391034526
Epoch 16, Loss: 48.95992329259072
Epoch 17, Loss: 48.40305864887853
Epoch 18, Loss: 48.43785679971018
Epoch 19, Loss: 48.131832354145665
Epoch 20, Loss: 48.06270054971018
Epoch 21, Loss: 47.91488832535282
Epoch 22, Loss: 48.03537881174395
Epoch 23, Loss: 47.85668138073337
Epoch 24, Loss: 47.68369561964466
Epoch 25, Loss: 47.79577652469758
Epoch 26, Loss: 47.45592458417339
Epoch 27, Loss: 

[I 2024-07-18 11:31:06,122] Trial 477 finished with value: 100.86985344434306 and parameters: {'latent_dim': 60, 'beta': 1.3311866773137568, 'learning_rate': 0.005, 'batch_size': 48, 'epochs': 905}. Best is trial 221 with value: 99.45155679744525.


Epoch 902, Loss: 47.31183711882561
Epoch 903, Loss: 47.45364635836694
Epoch 904, Loss: 47.30594888010333
Epoch 0, Loss: 234.2419254032258
Epoch 1, Loss: 224.06725554435485
Epoch 2, Loss: 219.1014289314516
Epoch 3, Loss: 210.31452368951614
Epoch 4, Loss: 206.19496471774193
Epoch 5, Loss: 200.79272933467743
Epoch 6, Loss: 200.27469758064515
Epoch 7, Loss: 192.605625
Epoch 8, Loss: 189.42477822580645
Epoch 9, Loss: 185.42605846774194
Epoch 10, Loss: 181.92183467741935
Epoch 11, Loss: 177.65218497983872
Epoch 12, Loss: 174.14220010080646
Epoch 13, Loss: 172.32505040322582
Epoch 14, Loss: 168.35216481854837
Epoch 15, Loss: 164.76737399193547
Epoch 16, Loss: 160.78218497983872
Epoch 17, Loss: 154.64053931451613
Epoch 18, Loss: 150.68253276209677
Epoch 19, Loss: 144.1584627016129
Epoch 20, Loss: 137.94824974798388
Epoch 21, Loss: 141.06321320564516
Epoch 22, Loss: 136.07522177419355
Epoch 23, Loss: 128.01151083669356
Epoch 24, Loss: 124.95430065524194
Epoch 25, Loss: 124.85711693548387
Epoch 

...

Epoch 802, Loss: 46.763495306199594
Epoch 803, Loss: 46.9178999968498
Epoch 804, Loss: 46.7133386624244
Epoch 0, Loss: 219.5940861265121
Epoch 1, Loss: 174.06705345892138
Epoch 2, Loss: 146.70032825100807
Epoch 3, Loss: 124.19075699344758
Epoch 4, Loss: 106.10035581527218
Epoch 5, Loss: 92.2007433688256
Epoch 6, Loss: 80.93963237147177
Epoch 7, Loss: 71.54729468560988
Epoch 8, Loss: 65.39163081999747
Epoch 9, Loss: 60.88663054435484
Epoch 10, Loss: 56.616131709929434
Epoch 11, Loss: 54.068509482106855
Epoch 12, Loss: 52.35066103043095
Epoch 13, Loss: 50.98909490769909
Epoch 14, Loss: 50.16696580456149
Epoch 15, Loss: 49.46627547725554
Epoch 16, Loss: 49.14825199250252
Epoch 17, Loss: 48.52069769090222
Epoch 18, Loss: 48.605372393208164
Epoch 19, Loss: 48.1212253890499
Epoch 20, Loss: 48.14067847467238
Epoch 21, Loss: 47.97095435357863
Epoch 22, Loss: 48.05202097246724
Epoch 23, Loss: 47.82939791771673
Epoch 24, Loss: 47.83915763608871
Epoch 25, Loss: 47.863733579574095
Epoch 26, Loss: 

[I 2024-07-18 11:42:12,076] Trial 498 finished with value: 101.18254055942062 and parameters: {'latent_dim': 51, 'beta': 1.4605849038276486, 'learning_rate': 0.005, 'batch_size': 48, 'epochs': 805}. Best is trial 221 with value: 99.45155679744525.


Epoch 802, Loss: 47.293494124873995
Epoch 803, Loss: 47.49174914944557
Epoch 804, Loss: 47.34132379347278
Epoch 0, Loss: 216.92419417842743
Epoch 1, Loss: 173.17949202998992
Epoch 2, Loss: 143.785978531376
Epoch 3, Loss: 121.7627253969254
Epoch 4, Loss: 104.76006639049899
Epoch 5, Loss: 91.60858823714717
Epoch 6, Loss: 79.91982744770665
Epoch 7, Loss: 71.33974404611895
Epoch 8, Loss: 65.29733422064012
Epoch 9, Loss: 60.33922327841482
Epoch 10, Loss: 56.34178482547883
Epoch 11, Loss: 53.72034089119204
Epoch 12, Loss: 52.122239753969254
Epoch 13, Loss: 50.6343790952621
Epoch 14, Loss: 49.716043898059475
Epoch 15, Loss: 49.0718017578125
Epoch 16, Loss: 48.60304226783014
Epoch 17, Loss: 48.58120739352319
Epoch 18, Loss: 48.51572572769657
Epoch 19, Loss: 47.85347683814264
Epoch 20, Loss: 47.81389904391381
Epoch 21, Loss: 47.90297879126764
Epoch 22, Loss: 47.699403706212195
Epoch 23, Loss: 47.51173753307712
Epoch 24, Loss: 47.73214182207661
Epoch 25, Loss: 47.34777843844506
Epoch 26, Loss: 4

[I 2024-07-18 11:42:46,912] Trial 499 finished with value: 100.64805079550639 and parameters: {'latent_dim': 52, 'beta': 1.21280304225261, 'learning_rate': 0.005, 'batch_size': 48, 'epochs': 805}. Best is trial 221 with value: 99.45155679744525.


Epoch 803, Loss: 47.112235383064515
Epoch 804, Loss: 47.03662432270665


In [6]:
# Save best hyperparameters
best_trial = study.best_trial
print(best_trial)
print(f"Best trial: {best_trial.values}")
print(f"Best hyperparameters: {best_trial.params}")

FrozenTrial(number=221, state=1, values=[99.45155679744525], datetime_start=datetime.datetime(2024, 7, 18, 9, 26, 51, 908729), datetime_complete=datetime.datetime(2024, 7, 18, 9, 27, 19, 888310), params={'latent_dim': 74, 'beta': 1.0078599067465557, 'learning_rate': 0.005, 'batch_size': 80, 'epochs': 805}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'latent_dim': IntDistribution(high=100, log=False, low=10, step=1), 'beta': FloatDistribution(high=10.0, log=False, low=1.0, step=None), 'learning_rate': CategoricalDistribution(choices=(0.005, 0.001, 0.0001, 1e-05, 1e-06)), 'batch_size': CategoricalDistribution(choices=(16, 48, 80, 112)), 'epochs': IntDistribution(high=905, log=False, low=5, step=100)}, trial_id=221, value=None)
Best trial: [99.45155679744525]
Best hyperparameters: {'latent_dim': 74, 'beta': 1.0078599067465557, 'learning_rate': 0.005, 'batch_size': 80, 'epochs': 805}
